In [1]:
import requests                     # To access API
from datetime import datetime
import pandas as pd

# Access WoW Auction House

In [2]:
# Create new access token to access API
def create_access_token(client_id, client_secret, region):
    data = {'grant_type':'client_credentials'}
    response = requests.post('https://%s.battle.net/oauth/token' % region,
                             data=data, auth=(client_id, client_secret))
    return response.json()

In [3]:
response = create_access_token('d16d7625352246078cfd3dbee36cac5e',
                               'mwNN2siP8otMNUCQz7KUaEFDNxxLdlN2',
                               'us')
token = response['access_token']

In [4]:
def get_illidan(token):
    search = 'https://us.api.blizzard' \
             '.com/data/wow/connected-realm/57/' \
             'auctions?namespace=dynamic-us&locale=en_GB&access_token=' + token
    response = requests.get(search)
    return response.json()['auctions']
illidan_auctions = get_illidan(token)

# Clean Data

In [5]:
df_auction = pd.DataFrame(illidan_auctions)

In [6]:
# Expand the item column
df_auction = df_auction.rename(columns={'id': 'auction_id'})
# Drop 'bonus_list' and 'modifiers'
df_auction = pd.concat([df_auction.drop(['item'], axis=1),
                        df_auction['item'].apply(pd.Series)], axis=1)

In [ ]:
# These are subgroups of an equipable item with the bonus stats (intellect agility, strength, etc)
# df_auction['Year'] = datetime.now().strftime('%Y')
# df_auction['Month'] = datetime.now().strftime('%m')
# df_auction['Day'] = datetime.now().strftime('%d')
# df_auction['Hour'] = datetime.now().strftime('%H')
# df_auction['Date'] = datetime.now().strftime('%m/%d/%Y')
# df_auction['Time'] = datetime.now().strftime('%H:%M:%S')

In [77]:
# filename = datetime.now().strftime('Illidan_US-%Y-%m-%d-%H-%M.csv')
# df_auction.to_csv(filename, index=False)

# Access Item Database

In [159]:
def get_illidan_item(token):
    search = 'https://us.api.blizzard' \
             '.com/data/wow/search/item?namespace=static-us&orderby=id' \
             '&_pageSize=1000&id=[1,]&_page=1&access_token=' + token
    response = requests.get(search)
    return response.json()['results']

item_ids = get_illidan_item(token)
# Create dataframe
df_items = pd.DataFrame(item_ids)

In [178]:
int(str(df_items['key'][999]).split('?')[0].split('item/')[1])

2428

In [158]:
x = 0
i = 1

while i < 2:
    def get_illidan_item(token):
        search = 'https://us.api.blizzard' \
                 '.com/data/wow/search/item?namespace=static-us&orderby=id' \
                 '&_pageSize=1000&id=[' + XXX + ',]&_page=1&access_token=' + \
                 token
        response = requests.get(search)
        return response.json()['results']

item_ids = get_illidan_item(token)
# Create dataframe
df_items = pd.DataFrame(item_ids)

KeyboardInterrupt: 

In [155]:
# Expand data dictionary
df_items = pd.concat([df_items.drop(['data'], axis=1),
                      df_items['data'].apply(pd.Series)], axis=1)
# Expand dictionaries
df_items = pd.concat([df_items.drop(['item_subclass'], axis=1),
                      df_items['item_subclass'].apply(pd.Series)
                     .rename(columns={'name':'item_subclass'})], axis=1)
df_items = pd.concat([df_items.drop(['item_class'], axis=1),
                      df_items['item_class'].apply(pd.Series)
                     .rename(columns={'name':'item_class'})], axis=1)
df_items = pd.concat([df_items.drop(['quality'], axis=1),
                      df_items['quality'].apply(pd.Series)
                     .rename(columns={'name':'quality'})], axis=1)
df_items = pd.concat([df_items.drop(['inventory_type'], axis=1),
                      df_items['inventory_type'].apply(pd.Series)
                     .rename(columns={'name':'inventory_type'})], axis=1)

In [156]:
# Choose the US value from the dictionary
df_items['item_id'] = df_items['media']\
    .apply(lambda x: x.get('id'))
df_items['name'] = df_items['name']\
    .apply(lambda x: x.get('en_US'))
df_items['item_subclass'] = df_items['item_subclass']\
    .apply(lambda x: x.get('en_US'))
df_items['item_class'] = df_items['item_class']\
    .apply(lambda x: x.get('en_US'))
df_items['quality'] = df_items['quality']\
    .apply(lambda x: x.get('en_US'))
df_items['inventory_type'] = df_items['inventory_type']\
    .apply(lambda x: x.get('en_US'))

In [157]:
df_items

,key,level,required_level,sell_price,is_equippable,purchase_quantity,media,max_count,is_stackable,name,...,id,item_subclass,id,item_class,id,quality,type,inventory_type,type,item_id
0,{'href': 'https://us.api.blizzard.com/data/wow...,1,1,3,True,1,{'id': 25},0,False,Worn Shortsword,...,25,Sword,7,Weapon,2,Common,COMMON,Main Hand,WEAPONMAINHAND,25
1,{'href': 'https://us.api.blizzard.com/data/wow...,1,1,4,True,1,{'id': 35},0,False,Bent Staff,...,35,Staff,10,Weapon,2,Common,COMMON,Two-Hand,TWOHWEAPON,35
2,{'href': 'https://us.api.blizzard.com/data/wow...,1,1,3,True,1,{'id': 36},0,False,Worn Mace,...,36,Mace,4,Weapon,2,Common,COMMON,Main Hand,WEAPONMAINHAND,36
3,{'href': 'https://us.api.blizzard.com/data/wow...,1,1,3,True,1,{'id': 37},0,False,Worn Axe,...,37,Axe,0,Weapon,2,Common,COMMON,Main Hand,WEAPONMAINHAND,37
4,{'href': 'https://us.api.blizzard.com/data/wow...,1,0,1,True,1,{'id': 38},0,False,Recruit's Shirt,...,38,Miscellaneous,0,Armor,4,Uncommon,UNCOMMON,Shirt,BODY,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,{'href': 'https://us.api.blizzard.com/data/wow...,26,20,118795,True,1,{'id': 2424},0,False,Brigandine Belt,...,2424,Mail,3,Armor,4,Common,COMMON,Waist,WAIST,2424
996,{'href': 'https://us.api.blizzard.com/data/wow...,26,20,237098,True,1,{'id': 2425},0,False,Brigandine Leggings,...,2425,Mail,3,Armor,4,Common,COMMON,Legs,LEGS,2425
997,{'href': 'https://us.api.blizzard.com/data/wow...,26,20,178126,True,1,{'id': 2426},0,False,Brigandine Boots,...,2426,Mail,3,Armor,4,Common,COMMON,Feet,FEET,2426
998,{'href': 'https://us.api.blizzard.com/data/wow...,26,20,111519,True,1,{'id': 2427},0,False,Brigandine Bracers,...,2427,Mail,3,Armor,4,Common,COMMON,Wrist,WRIST,2427


In [ ]:
df_items.iloc[999]['data']